# `Code` Efficient random walks


Tab set here (to compare algorithms):
````{tab-set}
```{tab-item} Tab 1 title
My first tab
```

```{tab-item} Tab 2 title
My second tab with `some code`!
```
````


```{toggle}
Some hidden toggle content!

```